## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-29-06-49-50 +0000,nyt,Fear and Hope in Venezuela as U.S. Warships Lurk,https://www.nytimes.com/2025/09/28/world/ameri...
1,2025-09-29-06-49-30 +0000,bbc,Mahmood to demand migrants earn right to settl...,https://www.bbc.com/news/articles/c0m4g3zvy02o...
2,2025-09-29-06-48-03 +0000,nypost,A raptor with no qualms about eating its oppon...,https://nypost.com/2025/09/29/world-news/a-rap...
3,2025-09-29-06-37-53 +0000,nyt,Moldova’s Pivotal Election Expected to End in ...,https://www.nytimes.com/2025/09/28/world/europ...
4,2025-09-29-06-33-30 +0000,bbc,Reeves to guarantee paid work for young people...,https://www.bbc.com/news/articles/c80gj2knrx4o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2351/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
185,trump,25
205,shooting,13
18,new,12
242,church,11
243,michigan,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
99,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,75
41,2025-09-29-01-58-54 +0000,nypost,President Trump shows off new White House blin...,https://nypost.com/2025/09/28/us-news/presiden...,63
35,2025-09-29-02-40-06 +0000,wapo,"At least 4 dead, 8 injured in Michigan church ...",https://www.washingtonpost.com/nation/2025/09/...,56
55,2025-09-28-23-53-00 +0000,wsj,"Eric Adams Ends Re-Election Bid, Reshaping Rac...",https://www.wsj.com/us-news/new-york-city-mayo...,55
45,2025-09-29-01-23-48 +0000,latimes,"After Michigan church shooting, Mayor Bass cal...",https://www.latimes.com/lifestyle/story/2025-0...,54


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
99,75,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
35,56,2025-09-29-02-40-06 +0000,wapo,"At least 4 dead, 8 injured in Michigan church ...",https://www.washingtonpost.com/nation/2025/09/...
55,55,2025-09-28-23-53-00 +0000,wsj,"Eric Adams Ends Re-Election Bid, Reshaping Rac...",https://www.wsj.com/us-news/new-york-city-mayo...
63,32,2025-09-28-23-26-36 +0000,nypost,Mississippi man shows up drunk to bail out pal...,https://nypost.com/2025/09/28/us-news/mississi...
77,26,2025-09-28-21-59-11 +0000,startribune,Three pedestrians struck and killed in Twin Ci...,https://www.startribune.com/man-killed-after-b...
140,24,2025-09-28-15-48-56 +0000,nypost,James Comey’s ex-No. 2 gives mealy-mouthed ans...,https://nypost.com/2025/09/28/us-news/comeys-e...
79,24,2025-09-28-21-46-12 +0000,nypost,30K runners retrace 9/11 hero’s path in NYC Tu...,https://nypost.com/2025/09/28/us-news/30k-runn...
182,23,2025-09-28-10-00-00 +0000,latimes,Barabak: Did Kamala Harris just destroy her 20...,https://www.latimes.com/politics/story/2025-09...
28,22,2025-09-29-03-50-42 +0000,nypost,Trump’s Portland troop plan sparks fierce lega...,https://nypost.com/2025/09/28/us-news/trumps-p...
189,22,2025-09-28-09-08-40 +0000,nypost,"Russia fires over 600 drones, missiles in dead...",https://nypost.com/2025/09/28/world-news/large...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
